In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
import json

# API Key
from config import gkey

In [2]:
# Import cities info from csv
cities_df = pd.read_csv("../Resources/KC_Metro_Cities.csv")
cities_df.head()

,Cities,State
0,Belton,Missouri
1,Gardner,Kansas
2,Gladstone,Missouri
3,Grandview,Missouri
4,Independence,Missouri


In [3]:
# add columns for restaurant name, restaurant address, rating
cities_df["Latitude"]=""
cities_df["Longitude"]=""
cities_df["Place Name"]=""
cities_df["Place Address"]=""
cities_df["Place Rating"]=""
cities_df["Place Latitude"]=""
cities_df["Place Longitude"]=""
cities_df.head()

,Cities,State,Latitude,Longitude,Place Name,Place Address,Place Rating,Place Latitude,Place Longitude
0,Belton,Missouri,,,,,,,
1,Gardner,Kansas,,,,,,,
2,Gladstone,Missouri,,,,,,,
3,Grandview,Missouri,,,,,,,
4,Independence,Missouri,,,,,,,


In [4]:
# create params dictionary to be updated with new city each iteration
params = {"key": gkey}

# loop through cities_df and run lat/lon search for each city
for index, row in cities_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    city = row["Cities"]
    state = row["State"]
    
    # update address key value
    params["address"] = f"{city},{state}"
    
    # make request
    cities_lat_lon = requests.get(base_url, params=params)
#     print(cities_lat_lon.url)
    
    # convert to json
    cities_lat_lon = cities_lat_lon.json()
    
    cities_df.loc[index, "Latitude"]=cities_lat_lon["results"][0]["geometry"]["location"]["lat"]
    cities_df.loc[index, "Longitude"]=cities_lat_lon["results"][0]["geometry"]["location"]["lng"]

#visualize to confirm
cities_df.head()

,Cities,State,Latitude,Longitude,Place Name,Place Address,Place Rating,Place Latitude,Place Longitude
0,Belton,Missouri,38.812,-94.5319,,,,,
1,Gardner,Kansas,38.8108,-94.9272,,,,,
2,Gladstone,Missouri,39.2039,-94.5547,,,,,
3,Grandview,Missouri,38.8858,-94.533,,,,,
4,Independence,Missouri,39.0911,-94.4155,,,,,


In [5]:
# params dictionary to update each iteration
params = {
    "radius": 15000,
    "types": "restaurant",
    "keyword": "chinese",
    "key": gkey
}
k=1
# use latitude/longitude to identify chinese retaurants
for index, row in cities_df.iterrows():
#     print(index)
#     get lat/lon from df
    lat = row["Latitude"]
    lon = row["Longitude"]
    
#     change location each iteration while 
#     leaving original params in place
    params["location"] = f"{lat},{lon}"
    
#     use the serach term "Chines Restaurant" and lat/lon
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#     make request and print url
    response = requests.get(base_url, params=params)
    
#     print(response.url)

#     convert to json
    response = response.json()
#     print(json.dumps(response, indent=4, sort_keys=True))

#     since some data may be missing, use try/except to skip any missing
#     data point
#     loop through to gather 5 indices
    pkey = []
    city_name = []
    state_name = []
    city_lat = []
    city_lon = []
    name = []
    address = []
    rating = []
    place_lat = []
    place_lon = []
    i = 0
    for responses in response:
        while i < 5:
            try:
                pkey.append(k)
                city_name.append(row["Cities"])
                state_name.append(row["State"])
                city_lat.append(lat)
                city_lon.append(lon)
                name.append(response["results"][i]["name"])
                address.append(response["results"][i]["vicinity"])
                rating.append(response["results"][i]["rating"])
                place_lat.append(response["results"][i]["geometry"]["location"]["lat"])
                place_lon.append(response["results"][i]["geometry"]["location"]["lng"])
            except (KeyError, IndexError):
                print("Missing field/result... skipping.")
        
            i+=1
            k+=1
#     if index = 0 create a new dataframe        
    if index == 0:
        chinese_restaurant_df = pd.DataFrame({"ID":pkey, "City":city_name,
                                             "State":state_name,
                                             "Latitude":city_lat,
                                             "Longitude":city_lon,
                                             "Place Name":name,
                                             "Place Address":address,
                                             "Place Rating":rating,
                                             "Place Latitude":place_lat,
                                             "Place Longitude":place_lon})
# otherwise create a new dataframe and then append the new dataframe to [0] df
    else:
        other_chinese_df = pd.DataFrame({"ID":pkey, "City":city_name,
                                        "State":state_name,
                                        "Latitude":city_lat,
                                        "Longitude":city_lon,
                                        "Place Name":name,
                                        "Place Address":address,
                                        "Place Rating":rating,
                                        "Place Latitude":place_lat,
                                        "Place Longitude":place_lon})
        chinese_restaurant_df = chinese_restaurant_df.append(other_chinese_df).reset_index(drop=True)

In [6]:
chinese_restaurant_df

,ID,City,State,Latitude,Longitude,Place Name,Place Address,Place Rating,Place Latitude,Place Longitude
0,1,Belton,Missouri,38.811953,-94.531898,China Dragon,"529 E Red Bridge Rd, Kansas City",3.8,38.925320,-94.585362
1,2,Belton,Missouri,38.811953,-94.531898,China Dragon J Chinese Restaurant,"436 SW Ward Rd, Lee's Summit",3.7,38.908375,-94.397390
2,3,Belton,Missouri,38.811953,-94.531898,Blue Koi,"10581 Mission Rd, Leawood",4.6,38.934567,-94.629679
3,4,Belton,Missouri,38.811953,-94.531898,China King,"3634 W 135th St, Overland Park",4.5,38.884138,-94.626852
4,5,Belton,Missouri,38.811953,-94.531898,Chinese Palace Restaurant,"10136 W 119th St, Overland Park",4.2,38.914108,-94.702678
5,6,Gardner,Kansas,38.810841,-94.927187,Master Wok,"14947 W 119th St, Olathe",4.4,38.911492,-94.759332
6,7,Gardner,Kansas,38.810841,-94.927187,Golden Palace Chinese Restaurant,"1711 S Mur-Len Rd, Olathe",4.4,38.856062,-94.780592
7,8,Gardner,Kansas,38.810841,-94.927187,Hong Kong Star Restaurant,"918 Old 56 Highway, Olathe",4.5,38.868724,-94.803250
8,9,Gardner,Kansas,38.810841,-94.927187,China Buffet,"2030 E Santa Fe St, Olathe",3.9,38.885063,-94.780751
9,10,Gardner,Kansas,38.810841,-94.927187,Asian Pearl Restaurant,"9532, 18138 W 119th St, Olathe",4.1,38.913049,-94.795692


In [18]:
chinese_restaurant_df.to_csv("../Output/chinese_restaurants.csv")

In [26]:
import sqlite3

In [31]:
conn = sqlite3.connect('../Output/places_db4.sqlite')

In [35]:
chinese_restaurant_df.to_sql('chinese', conn, if_exists='replace', index=False)

In [33]:
pd.read_sql('select * from chinese', conn)

,ID,City,State,Latitude,Longitude,Place Name,Place Address,Place Rating,Place Latitude,Place Longitude
0,1,Belton,Missouri,38.811953,-94.531898,China Dragon,"529 E Red Bridge Rd, Kansas City",3.8,38.925320,-94.585362
1,2,Belton,Missouri,38.811953,-94.531898,China Dragon J Chinese Restaurant,"436 SW Ward Rd, Lee's Summit",3.7,38.908375,-94.397390
2,3,Belton,Missouri,38.811953,-94.531898,Blue Koi,"10581 Mission Rd, Leawood",4.6,38.934567,-94.629679
3,4,Belton,Missouri,38.811953,-94.531898,China King,"3634 W 135th St, Overland Park",4.5,38.884138,-94.626852
4,5,Belton,Missouri,38.811953,-94.531898,Chinese Palace Restaurant,"10136 W 119th St, Overland Park",4.2,38.914108,-94.702678
5,6,Gardner,Kansas,38.810841,-94.927187,Master Wok,"14947 W 119th St, Olathe",4.4,38.911492,-94.759332
6,7,Gardner,Kansas,38.810841,-94.927187,Golden Palace Chinese Restaurant,"1711 S Mur-Len Rd, Olathe",4.4,38.856062,-94.780592
7,8,Gardner,Kansas,38.810841,-94.927187,Hong Kong Star Restaurant,"918 Old 56 Highway, Olathe",4.5,38.868724,-94.803250
8,9,Gardner,Kansas,38.810841,-94.927187,China Buffet,"2030 E Santa Fe St, Olathe",3.9,38.885063,-94.780751
9,10,Gardner,Kansas,38.810841,-94.927187,Asian Pearl Restaurant,"9532, 18138 W 119th St, Olathe",4.1,38.913049,-94.795692


In [36]:
#connect to the database
conn = sqlite3.connect('places_db4.sqlite')
c = conn.cursor()
c.executescript('''
   BEGIN TRANSACTION;
   ALTER TABLE chinese RENAME TO chinese_OLD;
   /*create a new table with the same column names and types while
   defining a primary key for the desired column*/
   CREATE TABLE chinese (  "ID" BIGINT PRIMARY KEY NOT NULL,
                           "City" TEXT,
                           "State" TEXT,
                           "Latitude"    FLOAT,
                           "Longitude"    FLOAT,
                           "Place Name" TEXT,
                           "Place Address"    TEXT,
                           "Place Rating"    TEXT,
                           "Place Latitude"    FLOAT,
                           "Place Longitude"    FLOAT
                           );
   INSERT INTO chinese SELECT * FROM chinese_OLD;
   DROP TABLE chinese_OLD;
   COMMIT TRANSACTION;
   ''')
#close out the connection
c.close()
conn.close()